In [86]:
import pandas as pd
import os
from utils import drop_categorical_columns
from utils import missing_values_table
import numpy as np

In [87]:
def drop_label_with_null(df, column_name):
    # Drop rows where the specified column is null
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

In [93]:
path = os.path.join('TestDataset', 'COVID.csv')
df = pd.read_csv(path)

df.shape

(75287, 135)

In [72]:
missing_values_table(df)

Your selected dataframe has 135 columns.
There are 77 columns that have missing values.


,Missing Values,% of Total Values,Data Type
geocoded_state,75287,100.0,float64
previous_day_admission_pediatric_covid_confirmed_12_17,37041,49.2,float64
previous_day_admission_pediatric_covid_confirmed_5_11,37028,49.2,float64
previous_day_admission_pediatric_covid_confirmed_0_4,36412,48.4,float64
previous_day_admission_pediatric_covid_confirmed_unknown,36296,48.2,float64
...,...,...,...
inpatient_beds_utilization_coverage,264,0.4,float64
inpatient_beds_utilization_numerator,264,0.4,float64
inpatient_beds_utilization_denominator,264,0.4,float64
inpatient_beds_used,264,0.4,float64


In [89]:
df.shape

(75287, 135)

In [90]:
# Replace all occurrences of -200 with NaN
df.replace('?', np.nan, inplace=True)

In [91]:
missing_values_table(df)

Your selected dataframe has 135 columns.
There are 77 columns that have missing values.


,Missing Values,% of Total Values,Data Type
geocoded_state,75287,100.0,float64
previous_day_admission_pediatric_covid_confirmed_12_17,37041,49.2,float64
previous_day_admission_pediatric_covid_confirmed_5_11,37028,49.2,float64
previous_day_admission_pediatric_covid_confirmed_0_4,36412,48.4,float64
previous_day_admission_pediatric_covid_confirmed_unknown,36296,48.2,float64
...,...,...,...
inpatient_beds_utilization_coverage,264,0.4,float64
inpatient_beds_utilization_numerator,264,0.4,float64
inpatient_beds_utilization_denominator,264,0.4,float64
inpatient_beds_used,264,0.4,float64


In [75]:
df.drop(columns='geocoded_state',inplace=True)

In [92]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(len(categorical_columns))
for col in categorical_columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')
# after those are taken care of we can drop the columns that are still object
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_columns)


2
['state', 'date']


In [77]:
# Convert date and time columns to datetime format
df['date'] = pd.to_datetime(df['date'])


# Extract year, month, day from the date column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day


# Drop the original date and time columns
df = df.drop(['date'], axis=1)

In [78]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(len(categorical_columns))

1


In [79]:
df['state']


0        NM
1        MA
2        RI
3        MA
4        RI
         ..
75282    CT
75283    DE
75284    OK
75285    NM
75286    TX
Name: state, Length: 75287, dtype: object

In [80]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Select categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Create a copy of the original dataframe to preserve the original columns
df_encoded = df.copy()

# Create an empty DataFrame to store the encoded columns
encoded_data_frames = []

# Define a list of transformers for each categorical column
for col in categorical_columns:

    # Create dummy variables using OneHotEncoder
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = encoder.fit_transform(df_encoded[[col]])

    # Get column names based on the original column and category
    column_names = [category for category in encoder.get_feature_names_out([col])]

    # Create a DataFrame for the encoded data
    encoded_df = pd.DataFrame(encoded_data, columns=column_names, index=df_encoded.index)

    # Add the encoded DataFrame to the list
    encoded_data_frames.append(encoded_df)

# Concatenate the original DataFrame and the list of encoded DataFrames
df_encoded = pd.concat([df_encoded] + encoded_data_frames, axis=1)

# Drop the original categorical columns
df_encoded.drop(categorical_columns, axis=1, inplace=True)

/Users/aryal/anaconda3/envs/Certain-Models/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [81]:
df_encoded.head(5)

,critical_staffing_shortage_today_yes,critical_staffing_shortage_today_no,critical_staffing_shortage_today_not_reported,critical_staffing_shortage_anticipated_within_week_yes,critical_staffing_shortage_anticipated_within_week_no,critical_staffing_shortage_anticipated_within_week_not_reported,hospital_onset_covid,hospital_onset_covid_coverage,inpatient_beds,inpatient_beds_coverage,...,state_TN,state_TX,state_UT,state_VA,state_VI,state_VT,state_WA,state_WI,state_WV,state_WY
0,16,38,1,18,36,1,2.0,54,4407.0,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8,70,1,6,72,1,4.0,78,17028.0,79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,9,1,4,9,1,8.0,13,2513.0,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12,89,1,10,91,1,5.0,101,19087.0,102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,9,2,4,9,2,7.0,14,2592.0,15,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
missing_values_table(df_encoded)

Your selected dataframe has 189 columns.
There are 76 columns that have missing values.


,Missing Values,% of Total Values,Data Type
previous_day_admission_pediatric_covid_confirmed_12_17,37041,49.2,float64
previous_day_admission_pediatric_covid_confirmed_5_11,37028,49.2,float64
previous_day_admission_pediatric_covid_confirmed_0_4,36412,48.4,float64
previous_day_admission_pediatric_covid_confirmed_unknown,36296,48.2,float64
staffed_icu_pediatric_patients_confirmed_covid,30149,40.0,float64
...,...,...,...
inpatient_beds_utilization_numerator,264,0.4,float64
inpatient_beds_utilization_denominator,264,0.4,float64
inpatient_beds_used,264,0.4,float64
inpatient_beds_utilization_coverage,264,0.4,float64


In [83]:
df_encoded['state_NM']

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
75282    0.0
75283    0.0
75284    0.0
75285    1.0
75286    0.0
Name: state_NM, Length: 75287, dtype: float64

In [84]:
path = os.path.join('Final-Datasets', 'COVID-encoded.csv')

In [85]:
df_encoded.to_csv(path,index=False)